In [1]:
import re
from copy import deepcopy

In [2]:
data = open("input/19").read().splitlines()

In [3]:
part_pattern = r"{x=(\d+),m=(\d+),a=(\d+),s=(\d+)}"

In [4]:
workflows = {}
parts = []

for line in data:
    if "=" in line:
        match = re.match(part_pattern, line)
        nums = list(map(int, match.groups()))
        parts.append({k: nums[idx] for idx, k in enumerate("xmas")})
    elif not line:
        continue
    else:
        name, other = line.split("{")
        workflows[name] = other[:-1]

In [5]:
def greater_than(a, b, equal=True):
    if equal:
        return a >= b
    return a > b
def less_than(a, b, equal=True):
    if equal:
        return a <= b
    return a < b

# Part 1

In [6]:
def check_condition(part, condition):
    condition, destination = condition.split(":")
    if "<" in condition:
        func = less_than
        split_char = "<"
    else:
        func = greater_than
        split_char = ">"
    
    variable, value = condition.split(split_char)
    if func(part[variable], int(value), equal=False):
        return destination

    return None

In [7]:
def check_part(part, current="in"):
    if current in ["R", "A"]:
        return current
    cur_workflow = workflows[current]
    valid_conditions = []
    for condition in cur_workflow.split(","):
        if ":" not in condition:
            continue
        if (res := check_condition(part, condition)):
            return check_part(part, current=res)

    default = cur_workflow.split(",")[-1]#["default"]
    if default in ["R", "A"]:
        return default
    else:
        return check_part(part, current=default)

In [8]:
part1 = 0
for part in parts:
    if check_part(part) == "A":
        part1 += sum(part.values())
print("Answer #1:", part1)

Answer #1: 402185


# Part 2
Got a lot of help from reddit on how to approach part 2

In [9]:
paths =  [["in", {'x': [0,4001], 'm':[0,4001], 'a':[0,4001], 's':[0,4001]}]]

valid_workflows = []
while paths:
    path = paths.pop()
    name, ranges = path
    if name == "R":
        continue
    if name == "A":
        valid_workflows.append(path)
        continue

    for entry in workflows[name].split(","):
        # Not new condition, add the default
        if ":" not in entry:
            paths.append([entry, deepcopy(ranges)])
            continue
        
        condition, destination = entry.split(":")
        if "<" in condition:
            func = less_than
            split_char = "<"
            range_idx = [1, 0]
            add_num = -1
        else:
            func = greater_than
            split_char = ">"
            range_idx = [0, 1]
            add_num = 1
            
        variable, value = condition.split(split_char)
        value = int(value)
        cur_range = ranges[variable]
        new_ranges = deepcopy(ranges)
        
        # Update current range with new value
        new_ranges[variable][range_idx[0]] = value
        
        # If valid condition
        if func(new_ranges[variable][range_idx[1]], value, equal=False):
            paths.append([destination, new_ranges])
            
        # Update range
        if new_ranges[variable][range_idx[1]] is not None:
            ranges[variable][range_idx[1]] = min(ranges[variable][1], value + add_num)

In [10]:
part2 = 0
for x, bounds in valid_workflows:
    num_ways = 1
    for key, bound in bounds.items():
        num_ways *= bound[1] - bound[0] - 1
    part2 += num_ways
print("Answer #2:", part2)

Answer #2: 130291480568730
